In [17]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [18]:
%cd drive/MyDrive/colab_projects/cocoapi/PythonAPI/

[Errno 2] No such file or directory: 'drive/MyDrive/colab_projects/cocoapi/PythonAPI/'
/Users/gbatch/Google Drive/colab_projects/cocoapi/PythonAPI


A lot of the code is from here:
* https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [19]:
import os

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets
from torchvision.io import read_image
from torchvision.transforms import ToTensor

In [20]:
simple_transforms = torch.nn.Sequential(
    torchvision.transforms.Resize(size=(256, 256)),
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
)

In [62]:
class CocoNoCropping(Dataset):
    def __init__(self, img_ids, my_annotations_file, img_dir,
                 transform=None, target_transform=None, divide_by_255=False):

        # dataDir, dataType, annFile, 
        # self.coco_datatype = coco_datatype
        # self.coco_annotations_file = 
        
        with open(img_ids, 'r') as f:
            self.img_ids = json.loads(f.read()) # python list saved as txt IDs as ints
        with open(my_annotations_file, 'r') as f:
            self.ids_to_labels = json.load(f)   # python dictionary saved as json IDs as strings
            
        self.img_dir = img_dir
        
        # target transforms
        self.target_transform = target_transform
        # image transforms
        self.transform = transform
        self.divide_by_255 = divide_by_255
        

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = str(self.img_ids[idx])
        img_name = (12-len(img_id)) * '0' + img_id + '.jpg'
        
        img_path = os.path.join(self.img_dir, img_name)
        image = read_image(img_path)
        
        # for b/w images just stack the same channel together 3 times
        if image.shape[0] == 1:
            image = image.repeat(3, 1, 1)
        
        if self.divide_by_255:
            image = image / 255.0
        
        label = self.ids_to_labels[img_id]
        if self.transform:
            image = self.transform(image)
            
        if self.target_transform:
            label = self.target_transform(label)
        sample = {"image": image, "label": label}
        return sample

In [63]:

    
# dev1_dataset = CocoNoCropping(
#     img_ids = '../my_splits/dev1_ids.txt',
#     my_annotations_file = '../my_annotations/imgIds_to_labels_train2017.json',
#     img_dir = '../my_images/dev1/',
#     transform=simple_transforms,
#     divide_by_255=True
# )

idx=1
with open('../my_splits/dev1_ids.txt', 'r') as f:
    img_ids = json.loads(f.read())
img_dir = '../my_images/dev1/'

    
img_id = str(img_ids[idx])
img_name = (12-len(img_id)) * '0' + img_id + '.jpg'

img_path = os.path.join(img_dir, img_name)
image = read_image(img_path)

print(image.shape)
print(image.shape[0])
print(image[0].shape)

if 1 == 1:
    image = torch.stack([image[0], image[0], image[0]], dim=0)
    
image.shape

torch.Size([3, 437, 583])
3
torch.Size([437, 583])


torch.Size([3, 437, 583])

In [22]:
%ls ../my_splits/

Icon?           dev_ids.txt     train1_ids.txt
dev1_ids.txt    test_ids.txt    train_ids.txt


In [23]:
%ls ../my_annotations/

Icon?                               strCatNms_to_imgIds_train2017.json
imgIds_to_labels_train2017.json     strCatNms_to_imgIds_val2017.json
imgIds_to_labels_val2017.json


In [24]:
%ls ../images/

train2017/ val2017/


In [25]:
train1_dataset = CocoNoCropping(
    img_ids = '../my_splits/train1_ids.txt',
    my_annotations_file = '../my_annotations/imgIds_to_labels_train2017.json',
    img_dir = '../my_images/train1/',
    transform=simple_transforms,
    divide_by_255=True
)

In [26]:
train1_dataloader = DataLoader(train1_dataset, batch_size=32, shuffle=True)

In [27]:
next(iter(train1_dataloader))

{'image': tensor([[[[ 1.3457e+00,  1.3412e+00,  1.3552e+00,  ...,  1.1229e+00,
             1.1270e+00,  1.1509e+00],
           [ 1.3578e+00,  1.3541e+00,  1.3365e+00,  ...,  1.1201e+00,
             1.1247e+00,  1.0970e+00],
           [ 1.3647e+00,  1.3351e+00,  1.3451e+00,  ...,  1.1296e+00,
             1.1262e+00,  1.0897e+00],
           ...,
           [ 7.2465e-01,  8.0136e-01,  8.0902e-01,  ...,  1.3150e+00,
             1.4698e+00,  1.3952e+00],
           [ 5.8511e-01,  4.9575e-01,  5.6002e-01,  ...,  1.1399e+00,
             1.3945e+00,  1.0495e+00],
           [ 3.4991e-01,  3.7774e-01,  5.4083e-01,  ...,  1.3153e+00,
             1.1551e+00,  9.2562e-01]],
 
          [[ 1.5927e+00,  1.5881e+00,  1.6024e+00,  ...,  1.3649e+00,
             1.3692e+00,  1.3672e+00],
           [ 1.6051e+00,  1.6014e+00,  1.5833e+00,  ...,  1.3621e+00,
             1.3668e+00,  1.3770e+00],
           [ 1.6121e+00,  1.5819e+00,  1.5921e+00,  ...,  1.3718e+00,
             1.3684e+00,  1.39

In [28]:
next(iter(train1_dataloader))['image'].min(), next(iter(train1_dataloader))['image'].max()

(tensor(-2.1179), tensor(2.6400))

In [29]:
for batch in train1_dataloader:
    inputs = batch['image'] # [batch size, channel, h, w]
    print(inputs.shape, inputs.dtype)
    
    # right now, `labels` is a list with num_classes tensors, each tensor with batch-size labels
    
    labels = torch.vstack(batch['label']).T.contiguous() 
    print(labels.shape, labels.dtype, labels.is_contiguous()) # labels
    print()
    #break

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 

In [65]:
dev1_dataset = CocoNoCropping(
    img_ids = '../my_splits/dev1_ids.txt',
    my_annotations_file = '../my_annotations/imgIds_to_labels_train2017.json',
    img_dir = '../my_images/dev1/',
    transform=simple_transforms,
    divide_by_255=True
)

In [66]:
dev1_dataloader = DataLoader(dev1_dataset, batch_size=32, shuffle=True)

In [67]:
for batch in dev1_dataloader:
    inputs = batch['image'] # [batch size, channel, h, w]
    print(inputs.shape, inputs.dtype)
    
    # right now, `labels` is a list with num_classes tensors, each tensor with batch-size labels
    
    labels = torch.vstack(batch['label']).T.contiguous() 
    print(labels.shape, labels.dtype, labels.is_contiguous()) # labels
    print()
    #break

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 256, 256]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 